In [6]:
%pip install pympi-ling

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob     # Import glob to easily loop over files
import pympi    # Import pympi to work with elan files
import string   # Import string to get the punctuation data
from cyrillicToLatin import cyrToIPA

In [26]:
def clean_text(string):
    
    if("сера") in string:
        print (string)
    
    a=re.compile("<.*?>")
    txt=a.sub('',string)
    

    
    if "нет в аудио" in txt:
        txt = txt.replace("[",'')
        txt = txt.replace("]",'')
        txt = txt.replace("нет в аудио",'')
    
    txt = txt.replace("'",'')
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).lower()
    txt = re.sub(r'{[^}]*}','',txt)
    
    spl = txt.split('/')
    if len(spl) > 2:
        txt = ''.join((spl[0], spl[-1]))
    
        
    txt = txt.replace('/','')
    
    
    txt = txt.replace('*', '')
    txt = txt.replace('"', '')
    txt = txt.replace('«', '')
    txt = txt.replace('»', '')
    txt = txt.replace('ʼ', '')
    txt = txt.replace('—','')
    txt = " ".join(txt.split())
    
    txt = txt.replace('ˊ','')
    txt = txt.replace('ˋ','')
#     txt = txt.replace('ң','ӈ')
#     txt = txt.replace('i','и')
#     txt = txt.replace('j','й')
#     txt = txt.replace('l','л')
#     txt = txt.replace('w','в')
#     txt = txt.replace('џ','')
#     txt = txt.replace('ӱ','у')
#     txt = txt.replace('ӓ','а̄')
#     txt = txt.replace('і','и')
#     txt = txt.replace('ӧ','о̄')
    txt = txt.replace('ѣ','е')
#     txt = txt.replace('йа','я')
    
#     txt = txt.replace('ӯ','у')
#     txt = txt.replace('ы̄','ы')
#     txt = txt.replace("'",'')
#     txt = txt.replace('x','х')
#     txt = txt.replace('c','с')
#     txt = txt.replace('e','е')
#     txt = txt.replace('y','у')
#     txt = txt.replace('u','и')
#     txt = txt.replace('o','о')
#     txt = txt.replace('p','р')
#     txt = txt.replace('a','а')
#     txt = txt.replace('k','к')
#     txt = txt.replace('n','п')
#     txt = txt.replace('s','с')
    txt = txt.replace("=",'')
    #txt =re.sub(r"\d+", "", txt)
    txt = re.sub(r'(?<=[.,])(?=[^\s])', r' ', txt)

    #txt = re.sub(r'[^\w\s]','',txt)
    txt = re.sub(r'[.,"\'?:!;]', '', txt)
    txt = txt.replace("[",'')
    txt = txt.replace("]",'')
    return txt

In [4]:
# Define some variables for later use
corpus_root = './evenki-corpus-master/corpus/evenki/eaf/'

#ort_tier_names = ['evCyr', 'rus']
ort_tier_names = ['ev', 'rus']

# Initialize the frequency dictionary
frequency_dict = {}
out = {}
#corpus = open("dataset/evn_corpus.tsv","w")
# Loop over all elan files the corpusroot subdirectory called eaf
for file_path in glob.glob('{}/*.eaf'.format(corpus_root)):
    #print(file_path)
    # Initialize the elan file
    eafob = pympi.Elan.Eaf(file_path)
    #print(eafob.get_tier_names())
    # Loop over all the defined tiers that contain orthography
    for ort_tier in ort_tier_names:
        # If the tier is not present in the elan file spew an error and
        # continue. This is done to avoid possible KeyErrors
        if ort_tier not in eafob.get_tier_names():
            if ort_tier == "ev":
                if "evCyr" in eafob.get_tier_names():
                    for annotation in eafob.get_annotation_data_for_tier("evCyr"):
                        if ort_tier not in out:
                            out[ort_tier] = [annotation[2]]
                        else:
                            out[ort_tier].extend([annotation[2]])
            print ('WARNING!!!',file_path)
            print ('One of the ortography tiers is not present in the elan file')
            print ('namely: {}. skipping this one...'.format(ort_tier))
        # If the tier is present we can loop through the annotation data
        else:
            #print("one")
            for annotation in eafob.get_annotation_data_for_tier(ort_tier):
                if "Вот умукэ̄н миндӯ бичо̄н ученикив" in annotation[2]:
                    print (file_path)
                if ort_tier not in out:
                    out[ort_tier] = [annotation[2]]
                else:
                    out[ort_tier].extend([annotation[2]])
                #corpus.write(thiselem+"\t"+nextelem+"\n")

./evenki-corpus-master/corpus/evenki/eaf/2005_Poligus_Khadonchina_LR2.eaf_new.eaf
WARNING!!! ./evenki-corpus-master/corpus/evenki/eaf/Arch_Levedeva_1952_Uchami_Lopoko_Robbery.eaf
One of the ortography tiers is not present in the elan file
namely: ev. skipping this one...


In [5]:
import re

In [24]:
corpus = open("dataset/evn_corpus.tsv","w",encoding='utf8')

In [27]:
i=0
while i<len(out["rus"]):
    if out['ev'][i] != '':
        if out['ev'][i][0].isdigit():
            i += 1
            continue

    # string1 = re.sub("^\s+|\n|\r|\s+$", '', string1)
    # string2 = re.sub("^\s+|\n|\r|\s+$", '', string2)

    

    string1 = clean_text(out['ev'][i])
    string1 = cyrToIPA(string1)
    string2 = clean_text(out['rus'][i])

    # if has_latin(string1):
    #     i+=1
    #     continue
    if string1 and string2:
        corpus.write(string1+"\t"+string2+"\n")
    i +=1

hаӈа̄рил эдо̄тын биhэ, мӯ эдо̄н чургӣдерэ, до̄скӣ нуӈан тар аӈит... сера сера как же это?... нютэт, нютэт.
Чтобы не было дыр, чтобы не просочилась вода, он вовнутрь этой самой... сера, сера {имеется в виду русское диалектное название смолы}, как же это? Смолой, смолой.
Говорила, что сейчас бисера очень много, только бы заниматься


In [28]:
corpus.close()